In [ ]:
import torch
import torch.nn as nn

class CustomYOLOv8Head(nn.Module):
    def __init__(self, num_classes, in_channels):
        super(CustomYOLOv8Head, self).__init__()
        # Define custom layers as required, e.g., with a modified output layer
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, num_classes + 5, kernel_size=1)  # 5: box coords and confidence

    def forward(self, x):
        x = self.conv1(x)
        return self.conv2(x)

# Assuming `model` is the YOLOv8 model object and we want to modify the head
model.model[-1] = CustomYOLOv8Head(num_classes=4, in_channels=model.model[-1].conv.out_channels)


In [ ]:
class CustomYOLOv8WithDense(nn.Module):
    def __init__(self, base_model, num_classes):
        super(CustomYOLOv8WithDense, self).__init__()
        self.base = base_model
        self.dense = nn.Linear(256, 128)
        self.output = nn.Linear(128, num_classes + 5)  # Modify as per your class count

    def forward(self, x):
        x = self.base(x)
        x = x.view(x.size(0), -1)  # Flatten for dense layer
        x = self.dense(x)
        return self.output(x)

# Applying this to the YOLOv8 model
model = CustomYOLOv8WithDense(model, num_classes=4)


In [ ]:
class SpatialAttentionLayer(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttentionLayer, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv(x)
        return self.sigmoid(x) * x

class YOLOv8WithAttention(nn.Module):
    def __init__(self, base_model, num_classes):
        super(YOLOv8WithAttention, self).__init__()
        self.base = base_model
        self.attention = SpatialAttentionLayer()
        self.output = nn.Conv2d(256, num_classes + 5, kernel_size=1)

    def forward(self, x):
        x = self.base(x)
        x = self.attention(x)
        return self.output(x)

model = YOLOv8WithAttention(model, num_classes=4)


In [ ]:
class CustomYOLOv8Loss(nn.Module):
    def __init__(self):
        super(CustomYOLOv8Loss, self).__init__()

    def forward(self, predictions, targets):
        # Example custom loss components
        box_loss = nn.MSELoss()(predictions[..., :4], targets[..., :4])  # Bounding box loss
        obj_loss = nn.BCEWithLogitsLoss()(predictions[..., 4], targets[..., 4])  # Objectness loss
        cls_loss = nn.CrossEntropyLoss()(predictions[..., 5:], targets[..., 5])  # Class loss

        # Custom weightings or adjustments
        total_loss = box_loss + obj_loss + 0.5 * cls_loss
        return total_loss

# Assuming `model` is wrapped in a training loop with `predictions` and `targets`
loss_fn = CustomYOLOv8Loss()
loss = loss_fn(predictions, targets)


In [ ]:
import torch
from ultralytics import YOLO  # Assuming YOLOv8 package is installed
from torch.utils.data import DataLoader
from ultralytics.yolo.v8.detect.train import train_one_epoch  # For YOLOv8 custom training

# Import your custom components here
from custom_components import CustomYOLOv8WithDense, CustomYOLOv8Loss  # Use the earlier defined custom classes

# Load and customize the YOLOv8 model with custom layers
base_model = YOLO('yolov8n.pt')  # Load a YOLOv8 base model
custom_model = CustomYOLOv8WithDense(base_model.model, num_classes=4)  # Modified model

# Configure the custom loss function
loss_fn = CustomYOLOv8Loss()

# Define the training loop
def train_custom_model(data_yaml='/content/extracted_files/Rabit_face_bird_cat_draft1/data.yaml', epochs=60, batch_size=16):
    # Load YOLO model with custom architecture
    yolo_model = YOLO(custom_model)

    # Setup data
    yolo_model.train(data=data_yaml, epochs=epochs, batch_size=batch_size, imgsz=640,
                     custom_loss_fn=loss_fn)  # Pass the custom loss function

# Run training
train_custom_model()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 21.5M/21.5M [00:00<00:00, 83.3MB/s]
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/extracted_files/Rabit_face_bird_cat_draft1/data.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, 